In [3]:
cd /content/drive/My Drive/yolov5/yolov5_new

/content/drive/My Drive/yolov5/yolov5_new


In [25]:
!python detect.py --weights /content/drive/MyDrive/yolov5/runs/train/vetable_yolov5s_results11/weights/best.pt --img 416 --conf 0.5 --source /content/drive/MyDrive/yolov5/inference/images/test6.jpg

detect: weights=['/content/drive/MyDrive/yolov5/runs/train/vetable_yolov5s_results11/weights/best.pt'], source=/content/drive/MyDrive/yolov5/inference/images/test6.jpg, imgsz=[416, 416], conf_thres=0.5, iou_thres=0.45, max_det=1000, device=, view_img=False, save_txt=False, save_conf=False, save_crop=False, nosave=False, classes=None, agnostic_nms=False, augment=False, visualize=False, update=False, project=runs/detect, name=exp, exist_ok=False, line_thickness=3, hide_labels=False, hide_conf=False, half=False
YOLOv5 🚀 2021-9-28 torch 1.9.0+cu102 CPU

Fusing layers... 
Model Summary: 224 layers, 7067395 parameters, 0 gradients, 16.4 GFLOPs
image 1/1 /content/drive/MyDrive/yolov5/inference/images/test6.jpg: 288x416 Done. (0.147s)
Speed: 1.8ms pre-process, 147.0ms inference, 0.3ms NMS per image at shape (1, 3, 416, 416)
Results saved to runs/detect/exp18


In [18]:
import torch
model = torch.load('/content/drive/MyDrive/yolov5/runs/train/vetable_yolov5s_results11/weights/best.pt')

In [30]:
import time
import cv2
import torch
import torch.backends.cudnn as cudnn
import numpy as np

from numpy import random
from models.experimental import attempt_load
from utils.datasets import letterbox
from utils.general import check_img_size, check_requirements, non_max_suppression, scale_coords
from utils.torch_utils import select_device, time_sync

In [43]:
SOURCE = '/content/drive/MyDrive/yolov5/inference/images/test8.jpg'
WEIGHTS = '/content/drive/MyDrive/yolov5/runs/train/vetable_yolov5s_results11/weights/best.pt'
IMG_SIZE = 640
DEVICE = ''
AUGMENT = False
CONF_THRES = 0.25
IOU_THRES = 0.45
CLASSES = None
AGNOSTIC_NMS = False

In [50]:
def detect():
    source, weights, imgsz = SOURCE, WEIGHTS, IMG_SIZE

    # Initialize
    device = select_device(DEVICE)
    half = device.type != 'cpu'  # half precision only supported on CUDA
    print('device:', device)

    # Load model
    model = attempt_load(weights, map_location=device)  # load FP32 model
    stride = int(model.stride.max())  # model stride
    imgsz = check_img_size(imgsz, s=stride)  # check img_size
    if half:
        model.half()  # to FP16

    # Get names and colors
    names = model.module.names if hasattr(model, 'module') else model.names
    colors = [[random.randint(0, 255) for _ in range(3)] for _ in names]

    # Run inference
    if device.type != 'cpu':
        model(torch.zeros(1, 3, imgsz, imgsz).to(device).type_as(next(model.parameters())))  # run once

    # Load image
    img0 = cv2.imread(source)  # BGR
    assert img0 is not None, 'Image Not Found ' + source

    # Padded resize
    img = letterbox(img0, imgsz, stride=stride)[0]

    # Convert
    img = img[:, :, ::-1].transpose(2, 0, 1)  # BGR to RGB, to 3x416x416
    img = np.ascontiguousarray(img)

    img = torch.from_numpy(img).to(device)
    img = img.half() if half else img.float()  # uint8 to fp16/32
    img /= 255.0  # 0 - 255 to 0.0 - 1.0
    if img.ndimension() == 3:
        img = img.unsqueeze(0)

    # Inference
    t0 = time_sync()
    pred = model(img, augment=AUGMENT)[0]
    print('pred shape:', pred.shape)

    # Apply NMS
    pred = non_max_suppression(pred, CONF_THRES, IOU_THRES, classes=CLASSES, agnostic=AGNOSTIC_NMS)

    # Process detections
    det = pred[0]
    print('det shape:', det.shape)

    s = ''
    s += '%gx%g ' % img.shape[2:]  # print string

    if len(det):
        # Rescale boxes from img_size to img0 size
        det[:, :4] = scale_coords(img.shape[2:], det[:, :4], img0.shape).round()

        # Print results
        for c in det[:, -1].unique():
            n = (det[:, -1] == c).sum()  # detections per class
            s += f"{n} {names[int(c)]}{'s' * (n > 1)}, "

    print(det[-1][-1])
    print(s)

In [51]:
if __name__ == '__main__':
    check_requirements(exclude=('pycocotools', 'thop'))
    with torch.no_grad():
            detect()

requirements: PyYAML>=5.3.1 not found and is required by YOLOv5, attempting auto-update...

requirements: 1 package updated per /content/drive/My Drive/yolov5/yolov5_new/requirements.txt
requirements: ⚠️ Restart runtime or rerun command for updates to take effect

device: cpu
pred shape: torch.Size([1, 17640, 11])
det shape: torch.Size([2, 6])
tensor(4.)
448x640 2 porks, 
